In [4]:
import pandas as pd
import re

## FEMA Data Prep
Convert place code to county FIPs.    
Limit to certain disaster types.    
Change year.

In [162]:
fema = pd.read_csv('femadeclarations.csv')

In [163]:
fema.columns

Index(['Disaster Number', 'IH Program Declared', 'IA Program Declared',
       'PA Program Declared', 'HM Program Declared', 'State ',
       'Declaration Date', 'Disaster Type', 'Incident Type', 'Title',
       'Incident Begin Date', 'Incident End Date', 'Disaster Close Out Date',
       'Place Code', 'Declared County/Area'],
      dtype='object')

In [164]:
fema['Place Code'] = fema['Place Code'] - 99000

In [165]:
fema.rename(columns = {'Place Code': 'CZ_FIPS', 'State ': 'STATE'}, inplace = True)

In [166]:
fema_event_types = ['Severe Storm(s)', 'Hurricane', 'Severe Ice Storm', 'Flood', 'Snow', 'Coastal Storm', 'Tornado', 'Freezing']
fema = fema[fema['Incident Type'].isin(fema_event_types)]

In [167]:
fema['Incident Begin Date'] = pd.to_datetime(fema['Incident Begin Date'])

In [168]:
fema2014 = fema[(fema['Incident Begin Date'] > '12-31-2013') & (fema['Incident Begin Date'] < '1-1-2015')]

In [169]:
fema2014['BEGIN_YEARMONTH'] = fema2014['Incident Begin Date'].map(lambda x: int(str(x.year)+str(("{:02}".format(x.month)))))

/Users/meghajain/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [170]:
fema2014.to_csv('fema2014.csv')

In [156]:
fema2014.head()

,Disaster Number,IH Program Declared,IA Program Declared,PA Program Declared,HM Program Declared,STATE,Declaration Date,Disaster Type,Incident Type,Title,Incident Begin Date,Incident End Date,Disaster Close Out Date,CZ_FIPS,Declared County/Area
1905,4207,No,No,Yes,Yes,VT,2/3/15,DR,Severe Storm(s),SEVERE WINTER STORM,2014-12-09,12/12/14,NaN,1.0,Addison (County)
1906,4207,No,No,Yes,Yes,VT,2/3/15,DR,Severe Storm(s),SEVERE WINTER STORM,2014-12-09,12/12/14,NaN,7.0,Chittenden (County)
1907,4207,No,No,Yes,Yes,VT,2/3/15,DR,Severe Storm(s),SEVERE WINTER STORM,2014-12-09,12/12/14,NaN,9.0,Essex (County)
1908,4207,No,No,Yes,Yes,VT,2/3/15,DR,Severe Storm(s),SEVERE WINTER STORM,2014-12-09,12/12/14,NaN,11.0,Franklin (County)
1909,4207,No,No,Yes,Yes,VT,2/3/15,DR,Severe Storm(s),SEVERE WINTER STORM,2014-12-09,12/12/14,NaN,15.0,Lamoille (County)


In [171]:
fema2014 = fema2014[['STATE', 'CZ_FIPS', 'BEGIN_YEARMONTH', 'Disaster Type']]

## NOAA Data Prep
Standardize FIPs by adding 0 in front of CZ_FIPS.    
Reduce to one line / episode.   
Reduce to counties only.    
Abbreviate states.

In [176]:
noaa2014 = pd.read_csv('/Users/meghajain/Downloads/StormEvents_details-ftp_v1.0_d2014_c20160617.csv')

In [177]:
states = {'AK': 'Alaska', 'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'}

def name_toabbrev(name):
    for k, v in states.items():
        if v == name:
            return (k)

In [178]:
noaa2014['STATE'] = noaa2014['STATE'].map(lambda x: x.title())
noaa2014['STATE'].head()

0    New Hampshire
1    New Hampshire
2    New Hampshire
3    New Hampshire
4    New Hampshire
Name: STATE, dtype: object

In [179]:
noaa2014['STATE'] = noaa2014['STATE'].map(name_toabbrev)
len(noaa2014)

59459

In [180]:
noaa2014['STATE'].head()

0    NH
1    NH
2    NH
3    NH
4    NH
Name: STATE, dtype: object

In [181]:
noaa2014 = noaa2014[noaa2014['CZ_TYPE'] == 'C']

In [182]:
noaa2014.columns

Index(['BEGIN_YEARMONTH', 'BEGIN_DAY', 'BEGIN_TIME', 'END_YEARMONTH',
       'END_DAY', 'END_TIME', 'EPISODE_ID', 'EVENT_ID', 'STATE', 'STATE_FIPS',
       'YEAR', 'MONTH_NAME', 'EVENT_TYPE', 'CZ_TYPE', 'CZ_FIPS', 'CZ_NAME',
       'WFO', 'BEGIN_DATE_TIME', 'CZ_TIMEZONE', 'END_DATE_TIME',
       'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
       'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'SOURCE',
       'MAGNITUDE', 'MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'CATEGORY', 'TOR_F_SCALE',
       'TOR_LENGTH', 'TOR_WIDTH', 'TOR_OTHER_WFO', 'TOR_OTHER_CZ_STATE',
       'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME', 'BEGIN_RANGE',
       'BEGIN_AZIMUTH', 'BEGIN_LOCATION', 'END_RANGE', 'END_AZIMUTH',
       'END_LOCATION', 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON',
       'EPISODE_NARRATIVE', 'EVENT_NARRATIVE', 'DATA_SOURCE'],
      dtype='object')

In [184]:
noaa2014[['STATE_FIPS', 'CZ_FIPS', 'CZ_NAME', 'STATE']].head()

,STATE_FIPS,CZ_FIPS,CZ_NAME,STATE
10,12,85,MARTIN,FL
11,35,13,DONA ANA,NM
17,12,3,BAKER,FL
20,12,107,PUTNAM,FL
21,12,1,ALACHUA,FL


In [185]:
def format_czfips(fips):
    if fips < 10:
        return '00{}'.format(fips)
    elif fips < 100:
        return '0{}'.format(fips)
    else:
        return fips
    
noaa2014['CZ_FIPS'] = noaa2014['CZ_FIPS'].map(format_czfips)

In [186]:
noaa2014['CZ_FIPS'].head()

10    085
11    013
17    003
20    107
21    001
Name: CZ_FIPS, dtype: object

In [187]:
noaa2014['FIPS'] = (noaa2014['STATE_FIPS'].astype('str') + noaa2014['CZ_FIPS'].astype('str')).astype('int')

In [188]:
noaa2014["LatLng"] = tuple(zip(noaa2014['BEGIN_LAT'], noaa2014['BEGIN_LON']))

In [189]:
noaa2014.to_csv('noaa2014.csv')

In [190]:
noaa2014 = noaa2014[['BEGIN_YEARMONTH', 'BEGIN_DAY', 'END_YEARMONTH',
       'END_DAY', 'EPISODE_ID', 'EVENT_ID', 'STATE', 'STATE_FIPS',
       'YEAR', 'MONTH_NAME', 'EVENT_TYPE', 'CZ_TYPE', 'CZ_FIPS', 'CZ_NAME',
       'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
       'DEATHS_INDIRECT', 'FIPS', 'LatLng']]

## Census Data

In [86]:
census = pd.read_csv('/Users/meghajain/Downloads/data/proj_3/census_info.csv')

In [87]:
census.columns

Index(['FIPS', 'State', 'County', '% Severe Housing Problems',
       '% Non-Hispanic White', '% Minority', '% Rural'],
      dtype='object')

In [88]:
census['FIPS'].head()

0    1001
1    1003
2    1005
3    1007
4    1009
Name: FIPS, dtype: int64

## PerCapita Income by Year 
Standardize FIPS by removing leading 0.

In [141]:
income = pd.read_csv('/Users/meghajain/Downloads/data/proj_3/incomepercapita_bycounty/CA1_1969_2014_ALL.csv')

In [142]:
income.columns

Index(['GeoFIPS', 'GeoName', 'Region', 'Table', 'LineCode',
       'IndustryClassification', 'Description', '1969', '1970', '1971', '1972',
       '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981',
       '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990',
       '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999',
       '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014'],
      dtype='object')

In [143]:
income['GeoFIPS'].head()

0    1000
1    1000
2    1000
3    1001
4    1001
Name: GeoFIPS, dtype: int64

In [144]:
income.rename(columns = {'GeoFIPS': 'FIPS'}, inplace = True)

In [145]:
income = income[income['LineCode'] == 3.0]

In [146]:
income = income[['FIPS', 'GeoName', '2014']]

## Merge all data but weather

In [191]:
merge1 = pd.merge(noaa2014, census, on = ['FIPS'], how = 'left')
len(noaa2014), len(merge1)

(32517, 32517)

In [192]:
merge2 = pd.merge(merge1, income, on = ['FIPS'], how = 'left')
len(merge2), len(merge1)

(32517, 32517)

In [193]:
merge3 = pd.merge(merge2, fema2014, on = ['CZ_FIPS', 'STATE', 'BEGIN_YEARMONTH'], how = 'left')
len(merge3), len(merge2)

(32535, 32517)

In [194]:
merge3.drop_duplicates(inplace = True)

## Final formatting
Drop nan lat/longs.    
Drop islands and Alaska.    
Add in Virginia grouped data.    
Add in duration of event.

In [200]:
merge3 = merge3[~((merge3['STATE'] == 'PR') | (merge3['STATE'] == 'AK') | (merge3['STATE'] == 'AS') | (merge3['STATE'] == 'GU') | (merge3['STATE'] == 'VI') | (merge3['STATE'] == 'HI'))]

In [201]:
merge3 = merge3[~(merge3['LatLng'].isnull())]

In [202]:
len(merge3)

32150

In [203]:
merge3.to_csv('/Users/meghajain/Desktop/data_noweather.csv')

In [204]:
data_noweather = pd.read_csv('data_noweather.csv')

In [205]:
data_noweather.columns

Index(['BEGIN_YEARMONTH', 'BEGIN_DAY', 'END_YEARMONTH', 'END_DAY',
       'EPISODE_ID', 'EVENT_ID', 'STATE', 'STATE_FIPS', 'YEAR', 'MONTH_NAME',
       'EVENT_TYPE', 'CZ_TYPE', 'CZ_FIPS', 'CZ_NAME', 'INJURIES_DIRECT',
       'INJURIES_INDIRECT', 'DEATHS_DIRECT', 'DEATHS_INDIRECT', 'FIPS',
       'LatLng', 'State', 'County', '% Severe Housing Problems',
       '% Non-Hispanic White', '% Minority', '% Rural', 'GeoName', '2014',
       'Disaster Type'],
      dtype='object')

In [206]:
data_noweather['DURATION'] = data_noweather['END_DAY'] - data_noweather['BEGIN_DAY'] + 1

In [210]:
data_noweather[['DURATION', 'BEGIN_DAY', 'END_DAY']].head(10)

,DURATION,BEGIN_DAY,END_DAY
0,1,11,11
1,1,11,11
2,1,8,8
3,1,18,18
4,1,10,10
5,1,30,30
6,1,7,7
7,2,7,8
8,1,13,13
9,1,23,23


## Add closest weather station ID.

In [214]:
noaa_withid = pd.read_csv('noaa_withid.csv', index_col=0)

In [220]:
data_withid = pd.merge(data_noweather, noaa_withid, on = ['LatLng'], how = 'left')
len(data_noweather), len(data_withid)

(32150, 32170)

In [221]:
data_withid.drop_duplicates(inplace = True)

In [414]:
data_withid.head()

,BEGIN_YEARMONTH,BEGIN_DAY,END_YEARMONTH,END_DAY,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,YEAR,MONTH_NAME,...,County,% Severe Housing Problems,% Non-Hispanic White,% Minority,% Rural,GeoName,2014,Disaster Type,DURATION,coop_id
0,201401,11,201401,11,81761,494246,AL,1,2014,January,...,Autauga,14,75.6,24.4,42.0,"Autauga, AL",36419.0,NaN,1,17366.0
1,201401,11,201401,11,81761,494247,AL,1,2014,January,...,Autauga,14,75.6,24.4,42.0,"Autauga, AL",36419.0,NaN,1,17366.0
2,201406,8,201406,8,87424,527520,AL,1,2014,June,...,Autauga,14,75.6,24.4,42.0,"Autauga, AL",36419.0,NaN,1,18438.0
3,201408,18,201408,18,89975,540932,AL,1,2014,August,...,Autauga,14,75.6,24.4,42.0,"Autauga, AL",36419.0,NaN,1,17366.0
4,201406,10,201406,10,87442,527724,AL,1,2014,June,...,Autauga,14,75.6,24.4,42.0,"Autauga, AL",36419.0,NaN,1,17366.0


In [ ]:
## STEP 1: Below code is to help with merge on weather. Creates a column with the day & month that is also in the weather data.
## STEP 2: Gives each event a unique id ( = index)
## STEP 3: Looks at missing values-- drops the one county that does not have income per capita info.
## STEP 4: Duplicates event based on duration

In [415]:
#step1
data_withid['MONTH'] = data_withid['BEGIN_YEARMONTH'].map(lambda x: x - 201400) 

In [416]:
data_withid['MONTH_DAY'] = tuple(zip(data_withid['MONTH'], data_withid['BEGIN_DAY']))

In [417]:
data_withid.to_csv('data_withid.csv') 

In [7]:
data_withid = pd.read_csv('data_withid.csv', index_col=0)

In [13]:
#step2
data_withid.reset_index(inplace = True)
data_withid.rename(columns = {'index': 'UNIQUE_ID'}, inplace = True)  

,UNIQUE_ID,BEGIN_YEARMONTH,BEGIN_DAY,END_YEARMONTH,END_DAY,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,YEAR,...,% Non-Hispanic White,% Minority,% Rural,GeoName,2014,Disaster Type,DURATION,coop_id,MONTH,MONTH_DAY
0,0,201401,11,201401,11,81761,494246,AL,1,2014,...,75.6,24.4,42.0,"Autauga, AL",36419.0,NaN,1,17366.0,1,"(1, 11)"
1,1,201401,11,201401,11,81761,494247,AL,1,2014,...,75.6,24.4,42.0,"Autauga, AL",36419.0,NaN,1,17366.0,1,"(1, 11)"
2,2,201406,8,201406,8,87424,527520,AL,1,2014,...,75.6,24.4,42.0,"Autauga, AL",36419.0,NaN,1,18438.0,6,"(6, 8)"
3,3,201408,18,201408,18,89975,540932,AL,1,2014,...,75.6,24.4,42.0,"Autauga, AL",36419.0,NaN,1,17366.0,8,"(8, 18)"
4,4,201406,10,201406,10,87442,527724,AL,1,2014,...,75.6,24.4,42.0,"Autauga, AL",36419.0,NaN,1,17366.0,6,"(6, 10)"
5,5,201404,30,201404,30,83782,523313,AL,1,2014,...,75.6,24.4,42.0,"Autauga, AL",36419.0,NaN,1,17366.0,4,"(4, 30)"
6,6,201404,7,201404,7,85674,521252,AL,1,2014,...,75.6,24.4,42.0,"Autauga, AL",36419.0,NaN,1,17366.0,4,"(4, 7)"
7,7,201404,7,201404,8,85674,524756,AL,1,2014,...,75.6,24.4,42.0,"Autauga, AL",36419.0,NaN,2,17366.0,4,"(4, 7)"
8,8,201410,13,201410,13,90875,544904,AL,1,2014,...,75.6,24.4,42.0,"Autauga, AL",36419.0,NaN,1,18380.0,10,"(10, 13)"
9,9,201412,23,201412,23,92364,553246,AL,1,2014,...,83.0,17.0,42.3,"Baldwin, AL",39040.0,NaN,1,12813.0,12,"(12, 23)"


In [17]:
#step3
data_withid.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32122 entries, 0 to 32149
Data columns (total 34 columns):
index                        32122 non-null int64
BEGIN_YEARMONTH              32122 non-null int64
BEGIN_DAY                    32122 non-null int64
END_YEARMONTH                32122 non-null int64
END_DAY                      32122 non-null int64
EPISODE_ID                   32122 non-null int64
EVENT_ID                     32122 non-null int64
STATE                        32104 non-null object
STATE_FIPS                   32122 non-null int64
YEAR                         32122 non-null int64
MONTH_NAME                   32122 non-null object
EVENT_TYPE                   32122 non-null object
CZ_TYPE                      32122 non-null object
CZ_FIPS                      32122 non-null int64
CZ_NAME                      32122 non-null object
INJURIES_DIRECT              32122 non-null int64
INJURIES_INDIRECT            32122 non-null int64
DEATHS_DIRECT                32122 n

In [16]:
# step3, removes county without percapita income
data_withid = data_withid[~(data_withid['2014'].isnull())] 

In [34]:
#step3, some latlngs were missed, fills in based on county
data_fillid = data_withid[['FIPS', 'coop_id']]
data_fillid = data_fillid[~(data_fillid['coop_id'].isnull())]
data_fillid.drop_duplicates(subset = ['FIPS'], inplace = True) 

In [35]:
data_fillid[data_fillid['FIPS'] == 5021]

,FIPS,coop_id
1187,5021,31632.0


In [36]:
data_withid2 = pd.merge(data_withid, data_fillid, on = ['FIPS'], how = 'left')
len(data_withid), len(data_withid2)

(32122, 32122)

In [38]:
data_withid2.loc[data_withid2['coop_id_x'].isnull(), 'coop_id_x'] = data_withid2['coop_id_y']

In [41]:
data_withid2.drop(labels = ['coop_id_y'], axis = 1, inplace = True) 

In [42]:
data_withid2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32122 entries, 0 to 32121
Data columns (total 34 columns):
index                        32122 non-null int64
BEGIN_YEARMONTH              32122 non-null int64
BEGIN_DAY                    32122 non-null int64
END_YEARMONTH                32122 non-null int64
END_DAY                      32122 non-null int64
EPISODE_ID                   32122 non-null int64
EVENT_ID                     32122 non-null int64
STATE                        32104 non-null object
STATE_FIPS                   32122 non-null int64
YEAR                         32122 non-null int64
MONTH_NAME                   32122 non-null object
EVENT_TYPE                   32122 non-null object
CZ_TYPE                      32122 non-null object
CZ_FIPS                      32122 non-null int64
CZ_NAME                      32122 non-null object
INJURIES_DIRECT              32122 non-null int64
INJURIES_INDIRECT            32122 non-null int64
DEATHS_DIRECT                32122 n

In [43]:
#removing the 3 events that did not have latlong
data_withid2 = data_withid2[~(data_withid2['coop_id_x'].isnull())] 

In [44]:
data_withid2.rename(columns = {'coop_id_x': 'coop_id'}, inplace = True)

In [45]:
#step4
data_formerge = pd.DataFrame([data_withid2.ix[idx] 
                       for idx in data_withid2.index 
                       for _ in range(data_withid2.ix[idx]['DURATION'])]).reset_index(drop=True) 

In [46]:
data_withid2.to_csv('data_withid_final.csv')
data_formerge.to_csv('data_formerge.csv')

## Formatting weather data.

In [47]:
df = pd.read_pickle('/Users/meghajain/Desktop/Metis/Projects/Project 3/OLD/initial_datasets/weather_data.p')

In [48]:
flag_columns = [column for column in df.columns if 'flag' in column]
df.drop(labels = df[flag_columns], inplace = True, axis = 1)

In [49]:
df[['coop_id', 'year', 'month']] = df[['coop_id', 'year', 'month']].convert_objects(convert_numeric = True)

/Users/meghajain/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


In [107]:
df_2014 = df[df['year'] == 2014]

In [51]:
df_2014.drop(labels = ['year'], axis = 1, inplace = True)

/Users/meghajain/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [52]:
df_2014[['day1', 'day2', 'day3', 'day4', 'day5',
       'day6', 'day7', 'day8', 'day9', 'day10', 'day11', 'day12', 'day13',
       'day14', 'day15', 'day16', 'day17', 'day18', 'day19', 'day20', 'day21',
       'day22', 'day23', 'day24', 'day25', 'day26', 'day27', 'day28', 'day29',
       'day30', 'day31']] = df_2014[['day1', 'day2', 'day3', 'day4', 'day5',
       'day6', 'day7', 'day8', 'day9', 'day10', 'day11', 'day12', 'day13',
       'day14', 'day15', 'day16', 'day17', 'day18', 'day19', 'day20', 'day21',
       'day22', 'day23', 'day24', 'day25', 'day26', 'day27', 'day28', 'day29',
       'day30', 'day31']].convert_objects(convert_numeric = True)

/Users/meghajain/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:9: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
/Users/meghajain/anaconda/lib/python3.5/site-packages/pandas/core/frame.py:2378: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [53]:
df_2014.columns

Index(['coop_id', 'month', 'element', 'day1', 'day2', 'day3', 'day4', 'day5',
       'day6', 'day7', 'day8', 'day9', 'day10', 'day11', 'day12', 'day13',
       'day14', 'day15', 'day16', 'day17', 'day18', 'day19', 'day20', 'day21',
       'day22', 'day23', 'day24', 'day25', 'day26', 'day27', 'day28', 'day29',
       'day30', 'day31'],
      dtype='object')

In [54]:
import numpy as np
df_2014.replace(to_replace = -9999, value = np.NaN, inplace = True)

/Users/meghajain/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [55]:
df_2014.reset_index

<bound method DataFrame.reset_index of          coop_id  month element  day1  day2  day3  day4   day5   day6   day7  \
4875      355162      1    TMAX  45.0  38.0  47.0  35.0   35.0   37.0   36.0   
4876      355162      1    TMIN  18.0  18.0  13.0   8.0    9.0   11.0   26.0   
4877      355162      1    PRCP   0.0   0.0   0.0   0.0    0.0    0.0  200.0   
4878      355162      1    SNOW   0.0   0.0   0.0   0.0    0.0    0.0    0.0   
4879      355162      1    SNWD   0.0   0.0   0.0   0.0    0.0    0.0    0.0   
4880      355162      2    TMAX  35.0  35.0  35.0  27.0   22.0   22.0   37.0   
4881      355162      2    TMIN  14.0  14.0  20.0  17.0   12.0    5.0   11.0   
4882      355162      2    PRCP   0.0   0.0  35.0   0.0    0.0    0.0    NaN   
4883      355162      2    SNOW   0.0   0.0   0.0   0.0    0.0    0.0    0.0   
6777      355362      1    TMAX  38.0  37.0  40.0  38.0   32.0   39.0   39.0   
6778      355362      1    TMIN  26.0  29.0  29.0  24.0   22.0   27.0   31.0   
6

In [56]:
df_2014.set_index(['coop_id', 'month', 'element'], inplace = True)

In [57]:
df_2014.T.fillna(df_2014.mean(axis = 1), inplace = True).T

day1       day2       day3       day4        day5  \
coop_id month element                                                           
355162  1     TMAX     45.000000  38.000000  47.000000  35.000000   35.000000   
              TMIN     18.000000  18.000000  13.000000   8.000000    9.000000   
              PRCP      0.000000   0.000000   0.000000   0.000000    0.000000   
              SNOW      0.000000   0.000000   0.000000   0.000000    0.000000   
              SNWD      0.000000   0.000000   0.000000   0.000000    0.000000   
        2     TMAX     35.000000  35.000000  35.000000  27.000000   22.000000   
              TMIN     14.000000  14.000000  20.000000  17.000000   12.000000   
              PRCP      0.000000   0.000000  35.000000   0.000000    0.000000   
              SNOW      0.000000   0.000000   0.000000   0.000000    0.000000   
355362  1     TMAX     38.000000  37.000000  40.000000  38.000000   32.000000   
              TMIN     26.000000  29.000000  29.000000  24.000000   22.000000   
              PRCP      0.000000   0.000000   0.000000   0.000000    0.000000   
              SNOW      0.000000   0.000000   0.000000   0.000000    0.000000   
              SNWD      0.000000   0.000000   0.000000   0.000000    0.000000   
        2     TMAX     39.000000  45.000000  42.000000  39.000000   35.000000   
              TMIN     31.000000  29.000000  31.000000  30.000000   19.000000   
              PRCP     78.416667  78.416667  78.416667  80.000000    0.000000   
        3     TMAX     54.000000  49.000000  51.000000  54.000000   53.000000   
              TMIN     37.000000  42.000000  42.000000  42.000000   44.000000   
              PRCP     47.263158  47.263158  47.263158  42.000000   72.000000   
              SNOW      0.000000   0.000000   0.000000   0.000000    0.000000   
              SNWD      0.000000   0.000000   0.000000   0.000000    0.000000   
        4     TMAX     49.000000  56.000000  58.000000  56.000000   48.000000   
              TMIN     35.000000  38.000000  34.000000  39.000000   41.000000   
              PRCP     11.000000   0.000000   0.000000  14.826087   14.826087   
              SNOW      0.000000   0.000000   0.000000   0.000000    0.000000   
              SNWD      0.000000   0.000000   0.000000   0.000000    0.000000   
        5     TMAX     88.000000  88.000000  73.000000  60.000000   56.000000   
              TMIN     44.000000  44.000000  47.000000  46.000000   42.000000   
              PRCP      0.000000   0.000000   0.000000   0.000000  114.000000   
...                          ...        ...        ...        ...         ...   
354003  7     TMAX     85.000000  94.000000  83.000000  74.000000   81.000000   
              TMIN     50.000000  56.000000  59.000000  57.000000   61.000000   
              PRCP      0.000000   0.000000   0.000000   0.000000    0.000000   
              SNOW      0.000000   0.000000   0.000000   0.000000    0.000000   
              SNWD      0.000000   0.000000   0.000000   0.000000    0.000000   
        8     TMAX     94.000000  94.000000  85.000000  95.000000   93.000000   
              TMIN     59.000000  63.000000  61.000000  62.000000   63.000000   
              PRCP      0.000000   0.000000   0.000000   0.000000    0.000000   
              SNOW      0.000000   0.000000   0.000000   0.000000    0.000000   
              SNWD      0.000000   0.000000   0.000000   0.000000    0.000000   
        9     TMAX     72.000000  80.000000  72.000000  70.000000   80.000000   
              TMIN     49.000000  50.000000  55.000000  44.000000   44.000000   
              PRCP      0.000000   0.000000   0.000000   0.000000    0.000000   
              SNOW      0.000000   0.000000   0.000000   0.000000    0.000000   
              SNWD      0.000000   0.000000   0.000000   0.000000    0.000000   
        10    TMAX     66.000000  65.000000  74.000000  77.000000   84.000000   
              TMIN     44.000000  44.000000  40.000000  40.

In [58]:
df_2014.head()

day1  day2  day3  day4  day5  day6   day7   day8  day9  \
coop_id month element                                                           
355162  1     TMAX     45.0  38.0  47.0  35.0  35.0  37.0   36.0   45.0  38.0   
              TMIN     18.0  18.0  13.0   8.0   9.0  11.0   26.0   30.0  29.0   
              PRCP      0.0   0.0   0.0   0.0   0.0   0.0  200.0  200.0   0.0   
              SNOW      0.0   0.0   0.0   0.0   0.0   0.0    0.0    0.0   0.0   
              SNWD      0.0   0.0   0.0   0.0   0.0   0.0    0.0    0.0   0.0   

                       day10  ...    day22  day23  day24  day25  day26  day27  \
coop_id month element         ...                                               
355162  1     TMAX      46.0  ...     45.0   40.0   37.0   39.0   53.0   48.0   
              TMIN      32.0  ...     13.0    9.0    8.0    7.0   13.0   23.0   
              PRCP       0.0  ...      0.0    0.0    0.0    0.0    0.0    0.0   
              SNOW       0.0  ...      0.0    0.0    0.0    0.0    0.0    0.0   
              SNWD       0.0  ...      0.0    0.0    0.0    0.0    0.0    0.0   

                       day28  day29  day30  day31  
coop_id month element                              
355162  1     TMAX      41.0   44.0   41.0   35.0  
              TMIN      25.0   26.0   22.0   18.0  
              PRCP       0.0    0.0    0.0    0.0  
              SNOW       0.0    0.0    0.0    0.0  
              SNWD       0.0    0.0    0.0    0.0  

[5 rows x 31 columns]

In [59]:
df_2014.columns.name = 'Days'

In [61]:
# df_2014.set_index(['coop_id', 'month', 'element'], inplace = True)

In [62]:
df_2014_step1 = df_2014.stack('Days').unstack('element')
df_2014_step1.head()

element              PRCP  SNOW  SNWD       TMAX       TMIN
coop_id month Days                                         
11084   1     day1   5.00   NaN   NaN  49.000000  44.000000
              day2  20.00   NaN   NaN  59.000000  48.000000
              day3  30.00   NaN   NaN  61.000000  28.000000
              day4   0.00   NaN   NaN  45.000000  25.000000
              day5  11.25   NaN   NaN  56.035714  30.678571

In [63]:
df_weather = df_2014_step1.reset_index()
df_weather.head()

element,coop_id,month,Days,PRCP,SNOW,SNWD,TMAX,TMIN
0,11084,1,day1,5.00,NaN,NaN,49.000000,44.000000
1,11084,1,day2,20.00,NaN,NaN,59.000000,48.000000
2,11084,1,day3,30.00,NaN,NaN,61.000000,28.000000
3,11084,1,day4,0.00,NaN,NaN,45.000000,25.000000
4,11084,1,day5,11.25,NaN,NaN,56.035714,30.678571


In [328]:
# df_2014_unstack.fillna(0, inplace = True) ## DECIDED NOT TO DO THIS BECAUSE IF IT DOESN'T HAVE INFO WOULD RATHER KNOW

In [67]:
import re
df_weather['Days'].replace(to_replace = r'[^0-9]', value = '', regex = True, inplace = True)

In [88]:
df_weather['MONTH_DAY'] = tuple(zip(df_weather['month'], df_weather['Days'].astype(int)))

In [89]:
df_weather.head()

element,coop_id,month,Days,PRCP,SNOW,SNWD,TMAX,TMIN,MONTH_DAY
0,11084,1,1,5.00,NaN,NaN,49.000000,44.000000,"(1, 1)"
1,11084,1,2,20.00,NaN,NaN,59.000000,48.000000,"(1, 2)"
2,11084,1,3,30.00,NaN,NaN,61.000000,28.000000,"(1, 3)"
3,11084,1,4,0.00,NaN,NaN,45.000000,25.000000,"(1, 4)"
4,11084,1,5,11.25,NaN,NaN,56.035714,30.678571,"(1, 5)"


In [90]:
df_weather.to_csv('df_weather.csv')

### MERGE DATA WITH WEATHER
Merge on coop_id, month, day    
Remove stations without any weather data and null values for temp.    
Group by unique_id in data df to get average/sum.    
Merge with original and drop duplicate unique ids

In [72]:
data_formerge = pd.read_csv('data_formerge.csv', index_col=0)

/Users/meghajain/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [75]:
data_formerge.columns

Index(['index', 'BEGIN_YEARMONTH', 'BEGIN_DAY', 'END_YEARMONTH', 'END_DAY',
       'EPISODE_ID', 'EVENT_ID', 'STATE', 'STATE_FIPS', 'YEAR', 'MONTH_NAME',
       'EVENT_TYPE', 'CZ_TYPE', 'CZ_FIPS', 'CZ_NAME', 'INJURIES_DIRECT',
       'INJURIES_INDIRECT', 'DEATHS_DIRECT', 'DEATHS_INDIRECT', 'FIPS',
       'LatLng', 'State', 'County', '% Severe Housing Problems',
       '% Non-Hispanic White', '% Minority', '% Rural', 'GeoName', '2014',
       'Disaster Type', 'DURATION', 'coop_id', 'MONTH', 'MONTH_DAY'],
      dtype='object')

In [76]:
data_formerge.rename(columns = {'index': 'UNIQUE_ID'}, inplace = True) 

In [93]:
data_formerge.rename(columns = {'BEGIN_DAY': 'DAY'}, inplace = True)

In [99]:
data_formerge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35813 entries, 0 to 35812
Data columns (total 34 columns):
UNIQUE_ID                    35813 non-null int64
BEGIN_YEARMONTH              35813 non-null int64
DAY                          35813 non-null int64
END_YEARMONTH                35813 non-null int64
END_DAY                      35813 non-null int64
EPISODE_ID                   35813 non-null int64
EVENT_ID                     35813 non-null int64
STATE                        35795 non-null object
STATE_FIPS                   35813 non-null int64
YEAR                         35813 non-null int64
MONTH_NAME                   35813 non-null object
EVENT_TYPE                   35813 non-null object
CZ_TYPE                      35813 non-null object
CZ_FIPS                      35813 non-null int64
CZ_NAME                      35813 non-null object
INJURIES_DIRECT              35813 non-null int64
INJURIES_INDIRECT            35813 non-null int64
DEATHS_DIRECT                35813 n

In [100]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 357647 entries, 0 to 357646
Data columns (total 9 columns):
coop_id      357647 non-null int64
MONTH        357647 non-null int64
DAY          357647 non-null object
PRCP         354702 non-null float64
SNOW         249736 non-null float64
SNWD         240312 non-null float64
TMAX         344906 non-null float64
TMIN         344720 non-null float64
MONTH_DAY    357647 non-null object
dtypes: float64(5), int64(2), object(2)
memory usage: 24.6+ MB


In [96]:
df_weather.rename(columns = {'Days': 'DAY', 'month': 'MONTH'}, inplace = True)
df_weather.columns

Index(['coop_id', 'MONTH', 'DAY', 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN',
       'MONTH_DAY'],
      dtype='object', name='element')

In [101]:
df_weather['DAY'] = pd.to_numeric(df_weather['DAY'], errors = 'coerce')

In [102]:
data_withweather = pd.merge(data_formerge, df_weather, on = ['coop_id', 'MONTH', 'DAY'], how = 'left')
len(data_withweather), len(data_formerge)

(35813, 35813)

In [104]:
data_withweather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35813 entries, 0 to 35812
Data columns (total 40 columns):
UNIQUE_ID                    35813 non-null int64
BEGIN_YEARMONTH              35813 non-null int64
DAY                          35813 non-null int64
END_YEARMONTH                35813 non-null int64
END_DAY                      35813 non-null int64
EPISODE_ID                   35813 non-null int64
EVENT_ID                     35813 non-null int64
STATE                        35795 non-null object
STATE_FIPS                   35813 non-null int64
YEAR                         35813 non-null int64
MONTH_NAME                   35813 non-null object
EVENT_TYPE                   35813 non-null object
CZ_TYPE                      35813 non-null object
CZ_FIPS                      35813 non-null int64
CZ_NAME                      35813 non-null object
INJURIES_DIRECT              35813 non-null int64
INJURIES_INDIRECT            35813 non-null int64
DEATHS_DIRECT                35813 n

In [110]:
#removing events with stations that did not report any weather
data_withinfo = data_withweather[~((data_withweather['PRCP'].isnull()) & (data_withweather['SNOW'].isnull()) & (data_withweather['SNWD'].isnull()) & (data_withweather['TMAX'].isnull()) & (data_withweather['TMIN'].isnull()))]

In [111]:
data_withinfo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29605 entries, 0 to 35812
Data columns (total 40 columns):
UNIQUE_ID                    29605 non-null int64
BEGIN_YEARMONTH              29605 non-null int64
DAY                          29605 non-null int64
END_YEARMONTH                29605 non-null int64
END_DAY                      29605 non-null int64
EPISODE_ID                   29605 non-null int64
EVENT_ID                     29605 non-null int64
STATE                        29589 non-null object
STATE_FIPS                   29605 non-null int64
YEAR                         29605 non-null int64
MONTH_NAME                   29605 non-null object
EVENT_TYPE                   29605 non-null object
CZ_TYPE                      29605 non-null object
CZ_FIPS                      29605 non-null int64
CZ_NAME                      29605 non-null object
INJURIES_DIRECT              29605 non-null int64
INJURIES_INDIRECT            29605 non-null int64
DEATHS_DIRECT                29605 n

In [123]:
#removing events that did not report any temp info because NA precip and snow can be filled in with 0
data_withinfo = data_withinfo[~((data_withinfo['TMAX'].isnull()) & data_withinfo['TMIN'].isnull())] 

In [124]:
data_withinfo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28844 entries, 0 to 35812
Data columns (total 40 columns):
UNIQUE_ID                    28844 non-null int64
BEGIN_YEARMONTH              28844 non-null int64
DAY                          28844 non-null int64
END_YEARMONTH                28844 non-null int64
END_DAY                      28844 non-null int64
EPISODE_ID                   28844 non-null int64
EVENT_ID                     28844 non-null int64
STATE                        28828 non-null object
STATE_FIPS                   28844 non-null int64
YEAR                         28844 non-null int64
MONTH_NAME                   28844 non-null object
EVENT_TYPE                   28844 non-null object
CZ_TYPE                      28844 non-null object
CZ_FIPS                      28844 non-null int64
CZ_NAME                      28844 non-null object
INJURIES_DIRECT              28844 non-null int64
INJURIES_INDIRECT            28844 non-null int64
DEATHS_DIRECT                28844 n

In [170]:
# replace null values with 0
data_withinfo[['PRCP', 'SNOW', 'SNWD']].fillna(0, inplace = True) 
data_withinfo[['PRCP', 'SNOW', 'SNWD']].replace('nan', 0, inplace = True) 

/Users/meghajain/anaconda/lib/python3.5/site-packages/pandas/core/frame.py:2762: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
/Users/meghajain/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [171]:
#calculate avg_temp
data_withinfo['AVG_TEMP'] = (data_withinfo[['TMAX', 'TMIN']].sum(axis = 1))/2
data_withinfo[['AVG_TEMP', 'TMAX', 'TMIN']].head()


,AVG_TEMP,TMAX,TMIN
0,56.5,68.0,45.0
1,56.5,68.0,45.0
2,77.5,88.0,67.0
3,82.0,92.0,72.0
4,80.0,89.0,71.0


In [172]:
data_withinfo.columns

Index(['UNIQUE_ID', 'BEGIN_YEARMONTH', 'DAY', 'END_YEARMONTH', 'END_DAY',
       'EPISODE_ID', 'EVENT_ID', 'STATE', 'STATE_FIPS', 'YEAR', 'MONTH_NAME',
       'EVENT_TYPE', 'CZ_TYPE', 'CZ_FIPS', 'CZ_NAME', 'INJURIES_DIRECT',
       'INJURIES_INDIRECT', 'DEATHS_DIRECT', 'DEATHS_INDIRECT', 'FIPS',
       'LatLng', 'State', 'County', '% Severe Housing Problems',
       '% Non-Hispanic White', '% Minority', '% Rural', 'GeoName', '2014',
       'Disaster Type', 'DURATION', 'coop_id', 'MONTH', 'MONTH_DAY_x', 'PRCP',
       'SNOW', 'SNWD', 'TMAX', 'TMIN', 'MONTH_DAY_y', 'AVG_TEMP'],
      dtype='object')

In [173]:
## GROUPING BY UNIQUEID
data_grouped = data_withinfo.groupby(by = 'UNIQUE_ID')
len(data_grouped)

25631

In [149]:
data_grouped['PRCP'].mean()

UNIQUE_ID
0         14.000000
1         14.000000
2         25.000000
3         55.000000
4          0.000000
5          3.000000
6        226.000000
7        226.000000
8        235.000000
9          2.000000
10         0.000000
11         0.000000
12         0.000000
13         0.000000
14         0.000000
15         0.000000
16         0.000000
17         0.000000
18       263.000000
19         0.000000
20         0.000000
21         0.000000
22       178.000000
23         4.000000
24         1.000000
25         1.000000
26       103.000000
27       103.000000
28       103.000000
29       266.000000
            ...    
32140     27.000000
32141     10.000000
32142      0.000000
32143      0.000000
32144      1.000000
32145     85.000000
32146      5.482759
32147      0.000000
32148      1.000000
32149      1.000000
32150      0.000000
32151      0.000000
32152      0.000000
32153     32.000000
32154      0.000000
32155      0.000000
32156      0.000000
32157      0.000000
32158     

In [179]:
## CREATING STATS BY EVENT
stats_df = pd.DataFrame({'AVG_TEMP_EVENT': data_grouped['AVG_TEMP'].mean(), 'AVG_PRCP': data_grouped['PRCP'].mean(), 'TOTAL_PRCP': data_grouped['PRCP'].sum(), 'AVG_SNOW': data_grouped['SNOW'].mean(), 'TOTAL_SNOW': data_grouped['SNOW'].sum()}).reset_index()

In [180]:
stats_df.head()

,UNIQUE_ID,AVG_PRCP,AVG_SNOW,AVG_TEMP_EVENT,TOTAL_PRCP,TOTAL_SNOW
0,0,14.0,10.0,56.5,14.0,10.0
1,1,14.0,10.0,56.5,14.0,10.0
2,2,25.0,NaN,77.5,25.0,NaN
3,3,55.0,NaN,82.0,55.0,NaN
4,4,0.0,NaN,80.0,0.0,NaN


In [181]:
stats_df.fillna(0, inplace=True)
stats_df.replace('nan', 0, inplace = True)
stats_df.head()

,UNIQUE_ID,AVG_PRCP,AVG_SNOW,AVG_TEMP_EVENT,TOTAL_PRCP,TOTAL_SNOW
0,0,14.0,10.0,56.5,14.0,10.0
1,1,14.0,10.0,56.5,14.0,10.0
2,2,25.0,0.0,77.5,25.0,0.0
3,3,55.0,0.0,82.0,55.0,0.0
4,4,0.0,0.0,80.0,0.0,0.0


In [167]:
stats_df['AVG_SNOW'].unique()

array([ 10.        ,   0.        ,  51.        ,  15.        ,
         1.        ,  20.        ,  12.        ,  13.        ,
        16.66666667,  28.        ,   1.77777778,  44.        ,
        32.        ,   8.33333333,  25.        ,  19.66666667,
        30.        ,   1.14285714,   2.33333333,   2.        ,
         8.5       ,  11.        ,   0.71428571,   0.14285714,
         8.7037037 ,   5.5       ,  10.5       ,   5.        ,
         8.        ,   3.        ,   0.90909091,  60.        ,
         1.36842105,   0.68181818,   6.        ,   3.28      ,
        73.        ,   7.5       ,   2.5       ,  13.30769231,
         7.        ,   0.35714286,   0.07407407,  20.75      ,
        70.        ,  42.        ,  48.        ,  10.2       ,
        21.        ,  26.        ,  17.14285714,  26.66666667,
        34.        ,  50.        ,  18.07142857,  25.71428571,
         8.63636364,   1.66666667,  11.85714286,   4.        ,
         3.36363636,  12.5       ,   1.25      ,   0.09

In [182]:
## REMERGE WITH MERGED DATASET AND REMOVE DUPLICATES OF UNIQUE ID
df_final = pd.merge(data_withinfo, stats_df, on = ['UNIQUE_ID'], how = 'left')
len(data_withinfo), len(df_final)

(28844, 28844)

In [183]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28844 entries, 0 to 28843
Data columns (total 46 columns):
UNIQUE_ID                    28844 non-null int64
BEGIN_YEARMONTH              28844 non-null int64
DAY                          28844 non-null int64
END_YEARMONTH                28844 non-null int64
END_DAY                      28844 non-null int64
EPISODE_ID                   28844 non-null int64
EVENT_ID                     28844 non-null int64
STATE                        28828 non-null object
STATE_FIPS                   28844 non-null int64
YEAR                         28844 non-null int64
MONTH_NAME                   28844 non-null object
EVENT_TYPE                   28844 non-null object
CZ_TYPE                      28844 non-null object
CZ_FIPS                      28844 non-null int64
CZ_NAME                      28844 non-null object
INJURIES_DIRECT              28844 non-null int64
INJURIES_INDIRECT            28844 non-null int64
DEATHS_DIRECT                28844 n

In [185]:
df_final.drop_duplicates(subset = ['UNIQUE_ID'], inplace = True)

In [186]:
df_final.columns

Index(['UNIQUE_ID', 'BEGIN_YEARMONTH', 'DAY', 'END_YEARMONTH', 'END_DAY',
       'EPISODE_ID', 'EVENT_ID', 'STATE', 'STATE_FIPS', 'YEAR', 'MONTH_NAME',
       'EVENT_TYPE', 'CZ_TYPE', 'CZ_FIPS', 'CZ_NAME', 'INJURIES_DIRECT',
       'INJURIES_INDIRECT', 'DEATHS_DIRECT', 'DEATHS_INDIRECT', 'FIPS',
       'LatLng', 'State', 'County', '% Severe Housing Problems',
       '% Non-Hispanic White', '% Minority', '% Rural', 'GeoName', '2014',
       'Disaster Type', 'DURATION', 'coop_id', 'MONTH', 'MONTH_DAY_x', 'PRCP',
       'SNOW', 'SNWD', 'TMAX', 'TMIN', 'MONTH_DAY_y', 'AVG_TEMP', 'AVG_PRCP',
       'AVG_SNOW', 'AVG_TEMP_EVENT', 'TOTAL_PRCP', 'TOTAL_SNOW'],
      dtype='object')

In [187]:
df_final.to_csv('df_final.csv')